In [1]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))
import pandas as pd
from configparser import ConfigParser

from new_repo.MyConfigParser import MyConfigParser
from new_repo.LoadData import DataLoader
from new_repo.QueryingData import Combine
from new_repo.FileGenerator import *

import new_repo.Accuracy
import new_repo.BuildModel
import new_repo.Optimization 


In [2]:

config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/Device type/uk/24 Sep Update/config ups UK 24 Sep.ini"
config_file = MyConfigParser(config_in_path)
config_dict = config_file.GetDict()

region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [3]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
cuts_df.drop(columns='Unnamed: 0', inplace=True)
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)
constraint_file

,market,account_type,funnel,Month,Year,variable_splits,Constrain_Variable_Name,periods_for_acc_cal,changepoint_prior_scale,seasonality_prior_scale,...,competitor_cost_split,new_confirmed,power,growth,MAPE,period_AE,periods,cutoff_month,MinWeight,MaxWeight
0,UK,Desktop,-,Apr,2024,UK_Desktop_-_Apr_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
1,UK,Desktop,-,May,2024,UK_Desktop_-_May_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
2,UK,Desktop,-,Jun,2024,UK_Desktop_-_Jun_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
3,UK,Desktop,-,Jul,2024,UK_Desktop_-_Jul_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
4,UK,Desktop,-,Aug,2024,UK_Desktop_-_Aug_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
5,UK,Desktop,-,Sep,2024,UK_Desktop_-_Sep_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
6,UK,Desktop,-,Oct,2024,UK_Desktop_-_Oct_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
7,UK,Desktop,-,Nov,2024,UK_Desktop_-_Nov_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,1.0,1.1
8,UK,Desktop,-,Dec,2024,UK_Desktop_-_Dec_2024,cost,183,0.1,10.0,...,na,na,na,linear,14.73,1.63,251.0,2024-04-25,0.8,1.2
9,UK,Mobile & Tablet,-,Apr,2024,UK_Mobile_&_Tablet_-_Apr_2024,cost,183,0.1,1.0,...,na,na,na,logistic,18.32,15.68,251.0,2024-04-25,0.8,1.2


In [4]:
#constraint_file.loc[(constraint_file['Month'] == 'Nov') & (constraint_file['MinWeight'] == 0.8) & (constraint_file['MaxWeight'] == 1.2), ['MinWeight', 'MaxWeight']] = [1.1, 1.3]

In [5]:
today = '06.06.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop(columns='Unnamed: 0')
# pos_df = pd.DataFrame({'variable_splits':analytical_file.columns[19:37],'pos': 1})
# pos_df.pos = 1
pos_df

,variable_splits,pos
0,UK_Desktop_-_Apr_2024,1.099064
1,UK_Desktop_-_May_2024,1.146488
2,UK_Desktop_-_Jun_2024,1.097007
3,UK_Desktop_-_Jul_2024,1.101416
4,UK_Desktop_-_Aug_2024,1.127207
5,UK_Desktop_-_Sep_2024,1.129210
6,UK_Desktop_-_Oct_2024,1.181382
7,UK_Desktop_-_Nov_2024,1.086705
8,UK_Desktop_-_Dec_2024,1.117834
9,UK_Mobile_&_Tablet_-_Apr_2024,0.849286


In [6]:
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1371850.0 difference between planned and forecast X:  0.0
yhat: 187888.0
analytical_spend_latest_period: 732585.7099999981
client_budget - analytical_spend_latest_period: 0


In [7]:
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1371850.0 difference between planned and forecast X:  0.0
yhat: 186042.0
analytical_spend_latest_period: 732585.7099999979
client_budget - analytical_spend_latest_period: 0


In [8]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

1371850.4 187888.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,UK,Desktop,-,2024-04-25,1540.0,21861.71,Apr,2024,UK_Desktop_-_Apr_2024
1,UK,Desktop,-,2024-04-26,3122.0,58079.05,Apr,2024,UK_Desktop_-_Apr_2024
2,UK,Desktop,-,2024-04-27,252.0,5132.65,Apr,2024,UK_Desktop_-_Apr_2024
3,UK,Desktop,-,2024-04-28,271.0,4413.95,Apr,2024,UK_Desktop_-_Apr_2024
4,UK,Desktop,-,2024-04-29,1170.0,12274.51,Apr,2024,UK_Desktop_-_Apr_2024
...,...,...,...,...,...,...,...,...,...
497,UK,Mobile & Tablet,-,2024-12-27,0.0,659.07,Dec,2024,UK_Mobile & Tablet_-_Dec_2024
498,UK,Mobile & Tablet,-,2024-12-28,17.0,659.07,Dec,2024,UK_Mobile & Tablet_-_Dec_2024
499,UK,Mobile & Tablet,-,2024-12-29,17.0,659.07,Dec,2024,UK_Mobile & Tablet_-_Dec_2024
500,UK,Mobile & Tablet,-,2024-12-30,4.0,124.66,Dec,2024,UK_Mobile & Tablet_-_Dec_2024


In [9]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

1371850.3000000003 186042.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,UK,Desktop,-,2024-04-25,1529.0,21636.65,Apr,2024
1,UK,Desktop,-,2024-04-26,3094.0,57481.14,Apr,2024
2,UK,Desktop,-,2024-04-27,250.0,5079.82,Apr,2024
3,UK,Desktop,-,2024-04-28,269.0,4368.51,Apr,2024
4,UK,Desktop,-,2024-04-29,1164.0,12148.15,Apr,2024
...,...,...,...,...,...,...,...,...
497,UK,Mobile & Tablet,-,2024-12-27,0.0,696.66,Dec,2024
498,UK,Mobile & Tablet,-,2024-12-28,17.0,696.66,Dec,2024
499,UK,Mobile & Tablet,-,2024-12-29,16.0,696.66,Dec,2024
500,UK,Mobile & Tablet,-,2024-12-30,5.0,131.77,Dec,2024


In [10]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.date.today().strftime("%Y-%m-%d")
forecast_n_base_df = forecast_n_base_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
forecast_n_base_df = forecast_n_base_df.drop(['Variable_splits'], axis = 1)
# forecast_n_base_df.insert(3, 'WeekStart', [ i for i in forecast_n_base_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
forecast_n_base_df.insert(0, 'Region', region)

forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'].sum(numeric_only=1)

OptimalConversionQuantity    144474.00
OptimalSpendValue            732585.75
BaseConversionQuantity       143935.00
BaseSpend                    732585.67
dtype: float64

In [11]:
daily_simulated_forecast[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,region,market,funnel,account_type,ds,X,Month,Year,UK_Desktop_-_Apr_2024,UK_Desktop_-_Aug_2024,...,UK_Mobile_&_Tablet_-_Jul_2024,UK_Mobile_&_Tablet_-_Jun_2024,UK_Mobile_&_Tablet_-_May_2024,UK_Mobile_&_Tablet_-_Nov_2024,UK_Mobile_&_Tablet_-_Oct_2024,UK_Mobile_&_Tablet_-_Sep_2024,Weekend,AOV,ROAS,yhat
0,EMEA,UK,-,Desktop,2024-04-25,21861.710300,Apr,2024,21861.710300,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,23.28,1.98,1540.0
1,EMEA,UK,-,Desktop,2024-04-26,58079.052844,Apr,2024,58079.052844,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,23.28,1.98,3122.0
2,EMEA,UK,-,Desktop,2024-04-27,5132.654587,Apr,2024,5132.654587,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,23.28,1.98,252.0
3,EMEA,UK,-,Desktop,2024-04-28,4413.954889,Apr,2024,4413.954889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,23.28,1.98,271.0
4,EMEA,UK,-,Desktop,2024-04-29,12274.514558,Apr,2024,12274.514558,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,23.28,1.98,1170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,EMEA,UK,-,Mobile & Tablet,2024-12-27,659.071368,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,21.90,0.85,0.0
498,EMEA,UK,-,Mobile & Tablet,2024-12-28,659.071368,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,21.90,0.85,17.0
499,EMEA,UK,-,Mobile & Tablet,2024-12-29,659.071368,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,21.90,0.85,17.0
500,EMEA,UK,-,Mobile & Tablet,2024-12-30,124.660513,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,21.90,0.85,4.0


In [12]:
forecast_n_base_df.loc[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,Region,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,UK,UK,Desktop,-,2024-04-25,1540.0,21861.71,Apr,2024,1529.0,21636.65,2024-06-06
1,UK,UK,Desktop,-,2024-04-26,3122.0,58079.05,Apr,2024,3094.0,57481.14,2024-06-06
2,UK,UK,Desktop,-,2024-04-27,252.0,5132.65,Apr,2024,250.0,5079.82,2024-06-06
3,UK,UK,Desktop,-,2024-04-28,271.0,4413.95,Apr,2024,269.0,4368.51,2024-06-06
4,UK,UK,Desktop,-,2024-04-29,1170.0,12274.51,Apr,2024,1164.0,12148.15,2024-06-06
...,...,...,...,...,...,...,...,...,...,...,...,...
497,UK,UK,Mobile & Tablet,-,2024-12-27,0.0,659.07,Dec,2024,0.0,696.66,2024-06-06
498,UK,UK,Mobile & Tablet,-,2024-12-28,17.0,659.07,Dec,2024,17.0,696.66,2024-06-06
499,UK,UK,Mobile & Tablet,-,2024-12-29,17.0,659.07,Dec,2024,16.0,696.66,2024-06-06
500,UK,UK,Mobile & Tablet,-,2024-12-30,4.0,124.66,Dec,2024,5.0,131.77,2024-06-06


In [13]:
forecast_n_base_df = forecast_n_base_df.rename(columns={'Account_type': 'device_segment'})
forecast_n_base_df = forecast_n_base_df.drop('Funnel', axis=1)

In [15]:
forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'].to_csv(base_path + 'UK Output.csv')

In [16]:
forecast_n_base_df[['Year']] = forecast_n_base_df[['Year']].astype(int)
forecast_n_base_df["SavedDate"] = pd.to_datetime(forecast_n_base_df["SavedDate"])
forecast_n_base_df['SavedDate'] = forecast_n_base_df['SavedDate'].dt.date

In [17]:
forecast_n_base_df.query("Ds >= '2024-06-01'", inplace=True)

# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'cost_by_device_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name

# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    forecast_n_base_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 428 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)